In [1]:
#All the imports here
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.algo.discovery.inductive import algorithm as inductive_miner
from pm4py.algo.discovery.inductive import algorithm as inductive_miner
from pm4py.visualization.process_tree import visualizer as pt_visualizer
from pm4py.objects.conversion.process_tree import converter as pt_converter
from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
from pm4py.visualization.dfg import visualizer as dfg_visualization
from pm4py.objects.conversion.dfg import converter as dfg_mining
from pm4py.visualization.petrinet import factory as pn_vis_factory
from collections import defaultdict 
import pandas as pd
from pm4py.objects.log.util import dataframe_utils
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.log.adapters.pandas import csv_import_adapter
from pm4py.objects.conversion.log import factory as conversion_factory
from pm4py.util import constants
import math
from datetime import date
import datetime
import numpy as np
from pm4py.objects.log.util import sorting
from pm4py.algo.filtering.log.attributes import attributes_filter
from pm4py.statistics.traces.log import case_statistics

/Users/deepak/opt/anaconda3/lib/python3.7/site-packages/pm4py/visualization/petrinet/__init__.py:20: UserWarning: please use the pm4py.visualization.petri_net package instead
  warnings.warn("please use the pm4py.visualization.petri_net package instead")


ImportError: cannot import name 'factory' from 'pm4py.visualization.petrinet' (/Users/deepak/opt/anaconda3/lib/python3.7/site-packages/pm4py/visualization/petrinet/__init__.py)

In [ ]:
log_csv4 = pd.read_csv('Production_Data.csv', sep=',')
log_csv4.rename(columns={'Activity': 'concept:name','Start Timestamp':'time:timestamp'}, inplace=True)
log_csv4['time:timestamp'] =  pd.to_datetime(log_csv4['time:timestamp'] , format='%m/%d/%Y %H:%M:%S')
log_csv4['Complete Timestamp'] =  pd.to_datetime(log_csv4['Complete Timestamp'] , format='%m/%d/%Y %H:%M:%S')
parameters = {log_converter.Variants.TO_EVENT_LOG.value.Parameters.CASE_ID_KEY: 'Case ID'}
event_log4 = log_converter.apply(log_csv4, parameters=parameters, variant=log_converter.Variants.TO_EVENT_LOG)
event_log4 = sorting.sort_timestamp(event_log4,"time:timestamp", False)
event_log4

In [ ]:
activities_dict={}

for case_index, case in enumerate(event_log4):
        #print("\n case index: %d  case id: %s" % (case_index, case.attributes["concept:name"]))
        
        for event_index, event in enumerate(case):
            if(event["concept:name"] in activities_dict.keys()): #and activities_dict[event["concept:name"]] is not None):
                #print(activities_dict[event["concept:name"]])
                activities_dict[event["concept:name"]].append((case.attributes["concept:name"],event_index, event["concept:name"], event['Complete Timestamp']-event['time:timestamp']))
            else:
                activities_dict[event["concept:name"]]=[(case.attributes["concept:name"],event_index,event["concept:name"],event['Complete Timestamp']-event['time:timestamp'])]

#activities_dict
average_dict={}
for k,v in activities_dict.items():
    i=0
    for l in v:
        i+=1
        if(k in average_dict.keys()):
            average_dict[k]=[average_dict[k][0]+l[3].total_seconds(), i]
        else:
            average_dict[k]=[l[3].total_seconds(),i]

average_dict2={}
for k,v in average_dict.items():
    average_dict2[k]= v[0]/v[1]
activities_dict.keys()  
#print(activities_duration,i)

In [ ]:
cols=['Detected Weakness Row','Case ID','Weakness Type (AF/PA)','Weakness ID','Weakness Origin', 'Weakness Time','Weakness Information','Weakness Measurement', 'Weakness Level']
df=pd.DataFrame(columns=cols)
df

In [ ]:
median_case_duration=case_statistics.get_median_caseduration (event_log4)
median_case_duration

In [ ]:
case_description_durations=case_statistics.get_cases_description (event_log4)
for k,v in case_description_durations.items():
    row={cols[0]:k, cols[1]: k, cols[2]:'PA', cols[3]:'10',cols[4]:'Automatic detection',cols[5]:'',cols[6]:abs(v['caseDuration']-median_case_duration),cols[7]:'Distance from median case duration in seconds',cols[8]:'Case level'}
    df=df.append(row, ignore_index=True)

In [ ]:
for case_index, case in enumerate(event_log4):
        #print("\n case index: %d  case id: %s" % (case_index, case.attributes["concept:name"]))
        
        for event_index, event in enumerate(case):
            row={cols[0]:case.attributes["concept:name"]+"-> Event "+str(event_index), cols[1]: case.attributes["concept:name"], cols[2]:'PA', cols[3]:'11',cols[4]:'Automatic detection',cols[5]:event['time:timestamp'],cols[6]: datetime.timedelta( abs(average_dict2[event["concept:name"]]-(event['Complete Timestamp']-event['time:timestamp']).total_seconds())),cols[7]:'Distance from average activity duration',cols[8]:'Activity level'}
            df=df.append(row, ignore_index=True)


In [ ]:
df.to_excel("PA_dataframe.xlsx")

In [ ]:
#df['Weakness Information'][0]


In [ ]:
#df.to_excel("AF_dataframe.xlsx")

In [ ]:


all_case_durations = case_statistics.get_all_casedurations(event_log4, parameters={
    case_statistics.Parameters.TIMESTAMP_KEY: "time:timestamp"})
         

In [ ]:
#all_case_durations
z=case_statistics.get_cases_description(event_log4)
z['Case 12']['caseDuration']

In [ ]:
median_case_duration = case_statistics.get_median_caseduration(event_log4, parameters={
    case_statistics.Parameters.TIMESTAMP_KEY: "time:timestamp"
})
median_case_duration

In [ ]:
from pm4py.statistics.traces.log import case_arrival
case_arrival_ratio = case_arrival.get_case_arrival_avg(event_log4, parameters={
    case_arrival.Parameters.TIMESTAMP_KEY: "time:timestamp"})
case_arrival_ratio      

In [ ]:
from pm4py.statistics.performance_spectrum import algorithm as performance_spectrum
ps = performance_spectrum.apply(event_log4, ["Final Inspection Q.C.","Packing"], parameters={performance_spectrum.Parameters.ACTIVITY_KEY: "concept:name",
                                            performance_spectrum.Parameters.TIMESTAMP_KEY: "time:timestamp"})
ps

In [ ]:
from pm4py.objects.log.util import interval_lifecycle
enriched_log = interval_lifecycle.assign_lead_cycle_time(event_log4)
enriched_log

In [ ]:
from pm4py.statistics.sojourn_time.log import get as soj_time_get

soj_time = soj_time_get.apply(event_log4, parameters={soj_time_get.Parameters.TIMESTAMP_KEY: "time:timestamp", soj_time_get.Parameters.START_TIMESTAMP_KEY: "start_timestamp"})
print(soj_time)

In [ ]:
import pm4py.algo.filtering.log.ltl  as ltl
import pm4py.algo.filtering.pandas.ltl as ltl2
import pandas as pd

from pm4py.objects.conversion.log import factory as conversion_factory
from pm4py.util import constants

from pm4py.objects.log.util import sorting

In [ ]:
log_csv = pd.read_csv('Production_Data.csv', sep=',')
log = conversion_factory.apply(log_csv, parameters={constants.PARAMETER_CONSTANT_CASEID_KEY: "Case ID",
                                               constants.PARAMETER_CONSTANT_ACTIVITY_KEY: "Activity",
                                                constants.PARAMETER_CONSTANT_START_TIMESTAMP_KEY:"Start Timestamp",
                                                constants.PARAMETER_CONSTANT_RESOURCE_KEY:"Resource",
                                                constants.PARAMETER_CONSTANT_TIMESTAMP_KEY:"Complete Timestamp"
                                               })
print("Log imported\n\n\n")

logSorted = sorting.sort_timestamp(log,"Start Timestamp", False)

In [ ]:
import os
from pm4py.algo.discovery.inductive import algorithm as inductive_miner

net, initial_marking, final_marking = inductive_miner.apply(logSorted)
from pm4py.visualization.petrinet import visualizer as pn_visualizer
gviz = pn_visualizer.apply(net, initial_marking, final_marking)
pn_visualizer.view(gviz)

In [ ]:
import os
from pm4py.objects.log.adapters.pandas import csv_import_adapter
from pm4py.objects.conversion.log import factory as conversion_factory
from pm4py.util import constants
dataframe = csv_import_adapter.import_dataframe_from_path('Production_Data.csv', sep=",")
log = conversion_factory.apply(dataframe, parameters={constants.PARAMETER_CONSTANT_CASEID_KEY: "Case ID",
                                               constants.PARAMETER_CONSTANT_ACTIVITY_KEY: "Activity",
                                                constants.PARAMETER_CONSTANT_START_TIMESTAMP_KEY:"Start Timestamp",
                                                constants.PARAMETER_CONSTANT_RESOURCE_KEY:"Resource",
                                                constants.PARAMETER_CONSTANT_TIMESTAMP_KEY:"Complete Timestamp"
                                               })

In [ ]:
A='Packing'
B='Final Inspection Q.C.'
print(ltl.ltl_checker.A_eventually_B(log, A, B))


In [ ]:
log

In [ ]:
ltl2.ltl_checker.attr_value_different_persons(dataframe, A)

In [ ]:

from pm4py.algo.organizational_mining.roles import algorithm as roles_discovery
roles = roles_discovery.apply(event_log4)
                                   